In [7]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../data/open-stax-texts-terms.parquet')

In [59]:
def return_duplicates(dataframe):
    print(f'Remaining Sections: {dataframe.shape[0]}')
    duplicates = (dataframe
                 .loc[dataframe.text.duplicated(keep=False)].book
                 .value_counts()
                 .sort_index()
                )
    print(f'Remaining duplicates: {duplicates.sum()}')
    display(duplicates)
    return dataframe

_ = return_duplicates(df)

Remaining Sections: 4892
Remaining duplicates: 1052


algebra-and-trigonometry-2e                 39
chemistry-2e                                43
chemistry-atoms-first-2e                    43
college-algebra-2e                          40
college-algebra-corequisite-support-2e       3
college-physics-2e                         232
college-physics-ap-courses-2e              232
introduction-business                        3
introductory-business-statistics           122
organizational-behavior                     10
principles-economics-2e                     78
principles-macroeconomics-2e                30
principles-macroeconomics-ap-courses-2e     24
principles-management                       13
principles-microeconomics-2e                76
principles-microeconomics-ap-courses-2e     64
Name: book, dtype: int64

In [60]:
drop_columns = ['college-physics-ap-courses-2e',
                'college-algebra-corequisite-support-2e',
                'principles-macroeconomics-ap-courses-2e',
                'principles-microeconomics-ap-courses-2e',
                'chemistry-atoms-first-2e',
                'introductory-business-statistics',
                'principles-macroeconomics-2e',
                'principles-microeconomics-2e',
                'college-algebra-2e',
                'principles-management',
               ]

deduplicated = df.loc[~df.book.isin(drop_columns)].pipe(return_duplicates)

Remaining Sections: 3855
Remaining duplicates: 0


Series([], Name: book, dtype: int64)

In [68]:
pd.Categorical(df.book).codes

array([ 0,  0,  0, ..., 39, 39, 39], dtype=int8)

In [81]:
np.concatenate((df.bold_term_offsets, df.no_bold_term_offsets))

array([array([array([2133, 2148]), array([2561, 2574]), array([2682, 2690]),
              array([3314, 3330]), array([5577, 5595]), array([7536, 7548]),
              array([8365, 8381]), array([9922, 9937]), array([10036, 10049]),
              array([10157, 10165]), array([10337, 10353]),
              array([10488, 10506]), array([11515, 11535]),
              array([11802, 11806]), array([11833, 11841]),
              array([12190, 12209]), array([17848, 17880]),
              array([18128, 18166]), array([18638, 18676]),
              array([19005, 19037]), array([19791, 19812]),
              array([21654, 21683]), array([21843, 21878]),
              array([22253, 22281]), array([22628, 22662]),
              array([26530, 26538]), array([26586, 26594]),
              array([26701, 26721]), array([30672, 30680]),
              array([31190, 31197])], dtype=object)                           ,
       array([array([23288, 23307]), array([24718, 24737]),
              array([26614,

In [96]:
def combine_offsets(row):
    return [(st, end, label) for label, values in
            {'bold_term': row.bold_term_offsets,
             'light_term': row.no_bold_term_offsets}
            .items()
            for (st, end) in values]

In [98]:
def convert_to_jsonl(dataframe):
    return (dataframe[['book', 'url', 'text', 'bold_term_offsets', 'no_bold_term_offsets']]
     .assign(labels=lambda x: 
             x.apply(lambda row: combine_offsets(row), axis=1))
     .rename(columns={'url': 'name',
                     })
     .assign(group= lambda x: pd.Categorical(x.book).codes)
     .drop(columns=['book', 'bold_term_offsets', 'no_bold_term_offsets'])
     .to_json(f'../assets/openstax.jsonl',
              orient='records',
              lines=True)
    )

convert_to_jsonl(deduplicated)

NameError: name 'labels_column' is not defined

In [71]:
convert_to_jsonl(deduplicated, labels_column='bold_term_offsets')
convert_to_jsonl(deduplicated, labels_column='no_bold_term_offsets')